# Creating the web scraper

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

# Getting the dataset

In [5]:
data = pd.read_csv('../data/music-subset.csv')

In [6]:
refined_data = data.loc[(data["rank"] == 1)]

In [7]:
# new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [8]:
new_data = refined_data.reset_index(drop = True)

In [9]:
refined_data.shape

(1000, 20)

In [10]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

KeyError: 'trend'

In [11]:
new_data.drop(index_names, inplace=True)

NameError: name 'index_names' is not defined

In [12]:
new_data.shape

(1000, 20)

In [13]:
new_data

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0
1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0
2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0
3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.0,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0
4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,1.0,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,PUSSY (feat. Lazza & Salmo),1,2020-05-10,Dark Polo Gang,https://open.spotify.com/track/7MhzmWFJYSEsL8w...,Italy,334344.0,0.852,0.751,9.0,-3.738,1.0,0.2800,0.2520,0.000010,0.1660,0.7000,150.010,161325.0
996,996,Karantene,1,2020-04-17,TIX,https://open.spotify.com/track/6sMMfGxa9tB59dM...,Norway,124824.0,0.628,0.754,0.0,-7.497,1.0,0.0348,0.2430,0.000000,0.3740,0.6470,104.989,132514.0
997,997,Gác Lại Âu Lo,1,2020-08-01,"Da LAB, Miu Lê",https://open.spotify.com/track/6HpDKKSDZ8QpSHf...,Vietnam,27084.0,0.745,0.527,6.0,-8.090,1.0,0.0786,0.5710,0.000000,0.1100,0.3850,95.034,282629.0
998,998,Medicina,1,2018-08-17,Anitta,https://open.spotify.com/track/4R5xvs5X0phd0EY...,Brazil,489983.0,0.798,0.784,4.0,-3.545,0.0,0.0667,0.0441,0.012900,0.1490,0.6530,93.960,141508.0


In [14]:
contain_values = data[data.artist.str.contains('The Weeknd')]
print (contain_values)

     index             title  rank        date                 artist  \
201    201   Blinding Lights     1  2020-05-18             The Weeknd   
249    249   Save Your Tears     1  2021-03-20             The Weeknd   
596    596       After Hours     1  2020-02-19             The Weeknd   
625    625  Call Out My Name     1  2018-04-12             The Weeknd   
943    943           Starboy     1  2017-01-05  The Weeknd, Daft Punk   

                                                   url     region  streams  \
201  https://open.spotify.com/track/0VjIjW4GlUZAMYd...  Lithuania   5165.0   
249  https://open.spotify.com/track/5QO79kh1waicV47...  Lithuania   6039.0   
596  https://open.spotify.com/track/4ejyZ4uJEXUURcg...    Romania  17107.0   
625  https://open.spotify.com/track/1gm616Plq4ScqNi...    Romania   5617.0   
943  https://open.spotify.com/track/5aAx2yezTd8zXrk...   Portugal  23933.0   

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
201         0.5

In [15]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [16]:
new_data.head()

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,1.0,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.0,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,1.0,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


In [17]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [18]:
new_data.shape

(1000, 20)

In [19]:
new_data.iloc[140087:140100,:]

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms


In [20]:
new_data

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0
1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0
2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0
3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.0,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0
4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,1.0,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,PUSSY (feat. Lazza & Salmo),1,2020-05-10,Dark Polo Gang,https://open.spotify.com/track/7MhzmWFJYSEsL8w...,Italy,334344.0,0.852,0.751,9.0,-3.738,1.0,0.2800,0.2520,0.000010,0.1660,0.7000,150.010,161325.0
996,996,Karantene,1,2020-04-17,TIX,https://open.spotify.com/track/6sMMfGxa9tB59dM...,Norway,124824.0,0.628,0.754,0.0,-7.497,1.0,0.0348,0.2430,0.000000,0.3740,0.6470,104.989,132514.0
997,997,Gác Lại Âu Lo,1,2020-08-01,"Da LAB, Miu Lê",https://open.spotify.com/track/6HpDKKSDZ8QpSHf...,Vietnam,27084.0,0.745,0.527,6.0,-8.090,1.0,0.0786,0.5710,0.000000,0.1100,0.3850,95.034,282629.0
998,998,Medicina,1,2018-08-17,Anitta,https://open.spotify.com/track/4R5xvs5X0phd0EY...,Brazil,489983.0,0.798,0.784,4.0,-3.545,0.0,0.0667,0.0441,0.012900,0.1490,0.6530,93.960,141508.0


# Import dataset

In [21]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [22]:
data.head(10)

,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0
1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0
2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0
3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.0,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0
4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,1.0,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0
5,7 rings,1,2019-03-16,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,Indonesia,125837.0,0.778,0.317,1.0,-10.732,0.0,0.3340,0.5920,0.000000,0.0881,0.3270,140.048,178627.0
6,Shape of You,1,2017-03-08,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Portugal,55132.0,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.9310,95.977,233713.0
7,China,1,2019-08-05,"Anuel AA, Daddy Yankee, KAROL G, J Balvin, Ozuna",https://open.spotify.com/track/2ksOAxtIxY8yElE...,Chile,579047.0,0.786,0.808,7.0,-3.702,1.0,0.0882,0.0846,0.000289,0.0822,0.6090,105.027,301714.0
8,Hawái,1,2020-09-20,Maluma,https://open.spotify.com/track/1yoMvmasuxZfqHE...,Spain,428210.0,0.783,0.727,6.0,-3.454,0.0,0.3890,0.1850,0.000025,0.0626,0.5580,180.067,199112.0


# Extracting artist and track name

In [23]:
track_nr = 0

In [24]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [25]:
get_track_name(track_nr)

'Chantaje'

In [26]:
get_track_artist(track_nr)

'Shakira'

# Getting URL

In [27]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [28]:
get_url(track_nr)

'https://genius.com/Shakira-Chantaje-lyrics'

# Scraping credits

In [29]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [30]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))
extLis

{'Feature': ['Maluma'],
 'Writer': ['Kevin ADG', 'Chan El Genio', 'Kenai', 'Maluma', 'Shakira'],
 'Label': [],
 'Producer': ['The Rudeboyz', 'Shakira', 'Maluma']}

# Dict to csv

In [31]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

### Path to neo4j import folder
Nicholas Stationær: C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv

Martins stationær: C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-e0579ea2-8404-4255-955c-aa57a88
28e4f\import\collaborators.csv

Martins bærbare: C:\Users\marti\.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import\collaborators.csv

In [33]:
df = pd.DataFrame.from_dict(credit_list)
df.to_csv (r'C:\Users\marti.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import\collaborators.csv', index = False, header=True)

OSError: Cannot save file into a non-existent directory: 'C:\Users\marti.Neo4jDesktop\relate-data\dbmss\dbms-a7f58857-34e9-48ad-b898-3dc31e11d74c\import'

In [34]:
df

,Feature,Writer,Label,Producer
0,Maluma,Kevin ADG,None,The Rudeboyz
1,None,Chan El Genio,None,Shakira
2,None,Kenai,None,Maluma
3,None,Maluma,None,None
4,None,Shakira,None,None


***

# Neo4j

In [35]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [36]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test1234")


In [37]:
conn.query('''MATCH (n) RETURN n;''')

[<Record n=<Node id=1 labels=frozenset() properties={}>>]

In [38]:
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'Chantaje'

In [39]:
artist

'Shakira'

### Query to insert into neo4j

In [40]:
def run_querys(artist, track):
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [41]:
run_querys(artist, track)

In [42]:
df

,Feature,Writer,Label,Producer
0,Maluma,Kevin ADG,None,The Rudeboyz
1,None,Chan El Genio,None,Shakira
2,None,Kenai,None,Maluma
3,None,Maluma,None,None
4,None,Shakira,None,None


### Getting the Shit

In [45]:
track_nr = 449
# For scraping
extLis = extract_credits(get_url(track_nr))
# Get credit
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
# Running neo4j query
run_querys(get_track_artist(track_nr), get_track_name(track_nr))

In [44]:
import time
tic = time.perf_counter()

for index in data.index:
    tac = time.perf_counter()
    print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + f'{tac - tic:0.0f} seconds elapsed', end='\r')
    artist = get_track_artist(index)
    track = get_track_name(index)
    # For scraping
    extLis = extract_credits(get_url(track_nr))
    # Get credit
    credit_list = pd.DataFrame.from_dict(extLis, orient='index').T
    # Running neo4j query
    run_querys(artist, track)
    tac = time.perf_counter()
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

KeyboardInterrupt: 